In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Local imports
from stability_testing import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
pv_dest_path = './data/hydro/npd/complementarity/pv_time_series'
wind_dest_path = '.data/hydro/npd/complementarity/wind_time_series'
pv_filename_prefix = 'cf_pv_profile'
wind_filename_prefix = 'cf_wind_profile'

c_stab_path = './data/hydro/npd/complementarity/c_stab'
cf_mix_path = './data/hydro/npd/complementarity/cf_mix'
pearsons_path = './data/hydro/npd/complementarity/pearson'

rep_year_df = pd.read_csv('./data/hydro/misc/representative_year.csv')
years = range(2007,2014)
months = range(1,13)
pv_wind_id_df = pd.read_csv('./data/hydro/misc/master_hydro_pv_wind.csv')

# build the base dataframe with the timestamps from 2012
rng = pd.date_range(f'2012-01-01 00:00:00+00:00', periods=8760, freq='1H')
base_df = pd.DataFrame()
base_df['dateTime'] = rng 

In [6]:
pv_wind_id_df.head(3)

,start,end,lat,lon,name,site_id,post_csv_filename,pv_id,wind_id,pv_wind_id_same
0,1/1/00,12/31/20,40.147715,-105.865879,GRANBY,9019500,./data/hydro//npd/synthetic_profiles/npd_90195...,CO01656,CO01656,True
1,1/1/00,12/31/20,36.466670,-91.530000,MAMMOTH SPRINGS DAM 3,7069220,NaN,AR01157,AR01157,True
2,1/1/00,12/31/20,35.394714,-106.547531,JEMEZ CANYON DAM,8328950,./data/hydro//npd/synthetic_profiles/npd_83289...,NM00003,NM00003,True


# 1. Create Montly Stability Coefficients

## Complementarity PV-Hydro

In [7]:
from pathlib import Path
from scipy.stats import pearsonr
import seaborn as sns

### Stability coefficient
missing_sites = []
df_missing = pd.DataFrame()

pv_wind_id_df.fillna(0, inplace=True)

pearsons_pv_hy = pd.DataFrame(columns=['site_id','p'])
annual_mean_p = pd.DataFrame()

annual_stab_df = pd.DataFrame(columns = ['site_id', 'lat', 'lon', 'c_stab'])

for idx, row in pv_wind_id_df.iterrows():
    
    if row['post_csv_filename'] == 0:
        print( f"No file for site {row['site_id']} ")
        missing_sites.append(row['site_id'])
        continue
    
    if row['site_id'] not in rep_year_df.site.values:
        print(f"Site doesn't have enough data: {row['site_id']}")
        empty_data_sites.append(row['site_id'])
        query_df.loc[idx,'post_csv_filename'] = ""
        continue
        
    print(f"{idx}. Processing PV profile {row['pv_id']} and NPD {row['site_id']}, {row['post_csv_filename']}")

    solar_df = pd.read_csv(f"{pv_dest_path}/{row['pv_id']}_{pv_filename_prefix}.csv",parse_dates=True,index_col=0)

    hydro_df = pd.read_csv(row['post_csv_filename'],parse_dates=True,index_col=0)
    hydro_df.fillna(0, inplace=True)
    
    if float(hydro_df['capacity_factor'].sum()) == 0.0:
        print( f"No data in site {row['site_id']} ")
        missing_sites.append(row['site_id'])
        continue

    c_stab = []
    pearsons = []
    
    # Finds the representative year for the given stream gage
    year = str(rep_year_df['year'].loc[rep_year_df.site == row['site_id']].values[0])
    
    # Massaging the timestamps of the hydropower time-series given that we are chosing the best year in terms of data completion.
    # Here, we copy the CF into a new dataframe with the time-stamps from 2012
    tmp_hydro_df_rep_year = hydro_df.loc[year,'capacity_factor'].copy().to_frame()
    tmp_hydro_df = base_df.copy()
    tmp_hydro_df.set_index('dateTime',inplace=True)
    tmp_hydro_df['capacity_factor'] = tmp_hydro_df_rep_year.loc[year,'capacity_factor'].values
     
    tmp_solar_df = solar_df.loc['2012-01-01':'2012-12-30'].copy()

    for month in months:
        
        #print(month)

        if month < 10:
            month = f'0{str(month)}'
        else:
            month = str(month)
        
        #tmp_solar_df = solar_df.loc['2012'].copy()
        
        tmp_hydro_monthly_df = tmp_hydro_df.loc[f'2012-{month}','capacity_factor'].copy().to_frame()
        tmp_solar_monthly_df = tmp_solar_df.loc[f'2012-{month}'].copy()

        tmp_hydro_monthly_df.rename(columns={'capacity_factor':row['site_id']}, inplace=True)
        tmp_solar_monthly_df.rename(columns={'pv_cf':row['site_id']}, inplace=True)

        # Compute complementarity metrics (PV as reference)
        _, c_stab_tmp = main_stability(tmp_solar_monthly_df, tmp_hydro_monthly_df)
        c_stab_tmp.rename(columns={row['site_id']:month}, inplace=True)

        c_stab.append(c_stab_tmp)
         
        corr, _ = pearsonr(tmp_solar_monthly_df[row['site_id']].values, tmp_hydro_monthly_df[row['site_id']].values)
        pearsons.append(corr)
        
    
    # ***************************************** Stability Coefficient *****************************************
    c_stab_df = pd.concat(c_stab, axis=1)
    
    annual_mean = c_stab_df.mean(axis=0)

    annual_stab_df.loc[idx,'site_id'] = row['site_id']
    annual_stab_df.loc[idx,'pv_id'] = row['pv_id']
    annual_stab_df.loc[idx,'lat'] = row['lat']
    annual_stab_df.loc[idx,'lon'] = row['lon']
    annual_stab_df.loc[idx,months] = annual_mean.values
    
    # ***************************************** Pearsons Coefficient *****************************************
    p_series = pd.Series(pearsons)

    annual_mean_p.loc[idx,'site_id'] = row['site_id']
    annual_mean_p.loc[idx,'pv_id'] = row['pv_id']
    annual_mean_p.loc[idx,'lat'] = row['lat']
    annual_mean_p.loc[idx,'lon'] = row['lon']
    annual_mean_p.loc[idx,months] = p_series.values

annual_stab_df.to_csv(f"{c_stab_path}/rep_year_MONTHLY_npd_stab_pv_hydro.csv")
annual_mean_p.to_csv(f"{c_stab_path}/rep_year_MONTHLY_npd_pearsons_pv_hydro.csv")

0. Processing PV profile CO01656 and NPD 9019500, ./data/hydro//npd/synthetic_profiles/npd_9019500_2016_synthetic.csv
No data in site 9019500 
No file for site 7069220 
2. Processing PV profile NM00003 and NPD 8328950, ./data/hydro//npd/synthetic_profiles/npd_8328950_2016_synthetic.csv
No data in site 8328950 
3. Processing PV profile PA00110 and NPD 3020000, ./data/hydro//npd/synthetic_profiles/npd_3020000_2019_synthetic.csv


/opt/anaconda3/envs/flexpower/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


4. Processing PV profile AL05903 and NPD 3592000, ./data/hydro//npd/synthetic_profiles/npd_3592000_2018_synthetic.csv


/opt/anaconda3/envs/flexpower/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


5. Processing PV profile NM00293 and NPD 7227000, ./data/hydro//npd/synthetic_profiles/npd_7227000_2015_synthetic.csv
No data in site 7227000 
6. Processing PV profile TX00001 and NPD 8063800, ./data/hydro//npd/synthetic_profiles/npd_8063800_2012_synthetic.csv
No data in site 8063800 
7. Processing PV profile WY01496 and NPD 6670500, ./data/hydro//npd/synthetic_profiles/npd_6670500_2019_synthetic.csv


...

89. Processing PV profile CO00976 and NPD 9246500, ./data/hydro//npd/synthetic_profiles/npd_9246500_2014_synthetic.csv
No data in site 9246500 
90. Processing PV profile 898 and NPD 1153000, ./data/hydro//npd/synthetic_profiles/npd_1153000_2018_synthetic.csv


/opt/anaconda3/envs/flexpower/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


91. Processing PV profile TX05020 and NPD 8048000, ./data/hydro//npd/synthetic_profiles/npd_8048000_2012_synthetic.csv
No data in site 8048000 
92. Processing PV profile OK02537 and NPD 7241000, ./data/hydro//npd/synthetic_profiles/npd_7241000_2013_synthetic.csv
No data in site 7241000 
93. Processing PV profile NY01211 and NPD 1500000, ./data/hydro//npd/synthetic_profiles/npd_1500000_2017_synthetic.csv
No data in site 1500000 
94. Processing PV profile PA00285 and NPD 3042500, ./data/hydro//npd/synthetic_profiles/npd_3042500_2015_synthetic.csv
No data in site 3042500 
95. Processing PV profile MI00361 and NPD 4151500, ./data/hydro//npd/synthetic_profiles/npd_4151500_2014_synthetic.csv
No data in site 4151500 
96. Processing PV profile WV07723 and NPD 3070260, ./data/hydro//npd/synthetic_profiles/npd_3070260_2015_synthetic.csv
No data in site 3070260 
No file for site 4101225 
98. Processing PV profile OH00005 and NPD 3124500, ./data/hydro//npd/synthetic_profiles/npd_3124500_2015_synth

/opt/anaconda3/envs/flexpower/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


106. Processing PV profile WI00296 and NPD 5430500, ./data/hydro//npd/synthetic_profiles/npd_5430500_2016_synthetic.csv
107. Processing PV profile RI00809 and NPD 1113895, ./data/hydro//npd/synthetic_profiles/npd_1113895_2017_synthetic.csv
108. Processing PV profile WI00589 and NPD 5378500, ./data/hydro//npd/synthetic_profiles/npd_5378500_2018_synthetic.csv


In [8]:
annual_stab_df

,site_id,lat,lon,c_stab,pv_id,1,2,3,4,5,6,7,8,9,10,11,12
3,3020000,41.475038,-79.445464,NaN,PA00110,0.627266,0.719977,0.354434,0.654553,0.484543,0.681588,0.148268,0.000000,0.000000,0.116790,0.248612,0.450875
4,3592000,34.398667,-87.987939,NaN,AL05903,0.082770,0.761249,0.734854,0.717576,0.251257,0.106416,0.000000,0.000000,0.000000,0.000000,0.312069,0.605100
7,6670500,42.170663,-104.69738,NaN,WY01496,0.000000,0.000000,0.000000,0.391029,0.682308,0.760980,0.631295,0.220805,0.000000,0.037039,0.735928,0.583629
8,1184000,41.988617,-72.603469,NaN,CT00583,0.230798,0.188899,0.674870,0.549485,0.230993,0.035257,0.000000,0.003683,0.005642,0.535416,0.536754,0.568094
10,3400800,36.751663,-83.25657,NaN,KY03061,0.444661,0.372895,0.807892,0.502291,0.000000,0.000000,0.226591,0.000000,0.000000,0.026260,0.111083,0.809101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,3111500,40.215532,-80.962672,NaN,OH01102,0.914751,0.894976,0.852882,0.815042,0.752256,0.689453,0.795540,0.729926,0.724512,0.813222,0.842302,0.948524
105,3056000,39.312938,-80.033924,NaN,WV09101,0.160210,0.669706,0.216095,0.257745,0.596866,0.109979,0.141580,0.000000,0.000000,0.262654,0.000783,0.915290
106,5430500,42.669398,-89.031526,NaN,WI00296,0.823221,0.568963,0.768328,0.799955,0.437772,0.281108,0.095220,0.148969,0.505929,0.807831,0.857497,0.873670
107,1113895,41.971454,-71.470843,NaN,RI00809,0.850447,0.560118,0.270129,0.621443,0.662716,0.340612,0.122577,0.021076,0.003514,0.172641,0.504026,0.183890


## Complementarity PV-Wind

In [9]:
from pathlib import Path
from scipy.stats import pearsonr

### Stability coefficient
missing_sites = []
df_missing = pd.DataFrame()

pv_wind_id_df.fillna(0, inplace=True)

annual_mean_p = pd.DataFrame()
pearsons_wind_hy = pd.DataFrame(columns=['site_id','p'])

annual_stab_df = pd.DataFrame(columns = ['site_id', 'lat', 'lon', 'c_stab'])

for idx, row in pv_wind_id_df.iterrows():
    
    if row['post_csv_filename'] == 0:
        print( f"No file for site {row['site_id']} ")
        missing_sites.append(row['site_id'])
        continue
    
    if row['site_id'] not in rep_year_df.site.values:
        print(f"Site doesn't have enough data: {row['site_id']}")
        empty_data_sites.append(row['site_id'])
        query_df.loc[idx,'post_csv_filename'] = ""
        continue

    print(f"{idx}. Processing Wind profile {row['wind_id']} and NPD {row['site_id']}, {row['post_csv_filename']}")

    wind_df = pd.read_csv(f"{wind_dest_path}/{row['wind_id']}_{wind_filename_prefix}.csv",parse_dates=True,index_col=0)

    hydro_df = pd.read_csv(row['post_csv_filename'],parse_dates=True,index_col=0)
    hydro_df.fillna(0, inplace=True)

    if float(hydro_df['capacity_factor'].sum()) == 0.0:
        print( f"No data in site {row['site_id']} ")
        missing_sites.append(row['site_id'])
        continue

    cf_mix = []
    c_stab = []
    pearsons = []
    
    # Finds the representative year for the given stream gage
    year = str(rep_year_df['year'].loc[rep_year_df.site == row['site_id']].values[0])
    
    # Massaging the timestamps of the hydropower time-series given that we are chosing the best year in terms of data completion.
    # Here, we copy the CF into a new dataframe with the time-stamps from 2012
    tmp_hydro_df_rep_year = hydro_df.loc[year,'capacity_factor'].copy().to_frame()
    tmp_hydro_df = base_df.copy()
    tmp_hydro_df.set_index('dateTime',inplace=True)
    tmp_hydro_df['capacity_factor'] = tmp_hydro_df_rep_year.loc[year,'capacity_factor'].values
    
    tmp_wind_df = wind_df.loc['2012-01-01':'2012-12-30'].copy()

    
    for month in months:

        if month < 10:
            month = f'0{str(month)}'
        else:
            month = str(month)
        
        tmp_hydro_monthly_df = tmp_hydro_df.loc[f'2012-{month}','capacity_factor'].copy().to_frame()
        tmp_wind_monthly_df = tmp_wind_df.loc[f'2012-{month}'].copy()

        tmp_hydro_monthly_df.rename(columns={'capacity_factor':row['site_id']}, inplace=True)
        tmp_wind_monthly_df.rename(columns={'wind_cf':row['site_id']}, inplace=True)

        # Compute complementarity metrics (PV as reference)
        cf_mix_tmp, c_stab_tmp = main_stability(tmp_wind_monthly_df, tmp_hydro_monthly_df)
        cf_mix_tmp.rename(columns={row['site_id']:month}, inplace=True)
        c_stab_tmp.rename(columns={row['site_id']:month}, inplace=True)

        cf_mix.append(cf_mix_tmp)
        c_stab.append(c_stab_tmp)
        
        corr, _ = pearsonr(tmp_wind_monthly_df[row['site_id']].values, tmp_hydro_monthly_df[row['site_id']].values)
        pearsons.append(corr)

    cf_mix_df = pd.concat(cf_mix, axis=1)
    #cf_mix_df.to_csv(f"{cf_mix_path}/hydro-pv/{row['site_id']}.csv")

    c_stab_df = pd.concat(c_stab, axis=1)
    #c_stab_df.to_csv(f"{c_stab_path}/hydro-pv/{row['site_id']}.csv")

    annual_mean = c_stab_df.mean(axis=0)

    annual_stab_df.loc[idx,'site_id'] = row['site_id']
    annual_stab_df.loc[idx,'pv_id'] = row['pv_id']
    annual_stab_df.loc[idx,'lat'] = row['lat']
    annual_stab_df.loc[idx,'lon'] = row['lon']
    annual_stab_df.loc[idx,months] = annual_mean.values
    
    # ***************************************** Pearsons Coefficient *****************************************
    p_series = pd.Series(pearsons)

    annual_mean_p.loc[idx,'site_id'] = row['site_id']
    annual_mean_p.loc[idx,'pv_id'] = row['pv_id']
    annual_mean_p.loc[idx,'lat'] = row['lat']
    annual_mean_p.loc[idx,'lon'] = row['lon']
    annual_mean_p.loc[idx,months] = p_series.values
    #break

annual_stab_df.to_csv(f"{c_stab_path}/rep_year_MONTHLY_npd_stab_wind_hydro.csv")
annual_mean_p.to_csv(f"{c_stab_path}/rep_year_MONTHLY_npd_pearsons_wind_hydro.csv")

0. Processing Wind profile CO01656 and NPD 9019500, ./data/hydro//npd/synthetic_profiles/npd_9019500_2016_synthetic.csv


FileNotFoundError: [Errno 2] No such file or directory: '.data/hydro/npd/complementarity/wind_time_series/CO01656_cf_wind_profile.csv'

# Relationship plots

## Ranking the sites based on their annual stability coefficient

### NPD PV-Hydro

In [ ]:
import pandas as pd

In [ ]:
pv_hydro_annual_stab_df = pd.read_csv(f"{c_stab_path}/rep_year_MONTHLY_npd_stab_pv_hydro.csv")
pv_hydro_annual_stab_df

# Cleaning up the dataframe
c_stab_pv_hydro_df = pd.DataFrame()
c_stab_pv_hydro_df['site_id'] = pv_hydro_annual_stab_df['site_id']
c_stab_pv_hydro_df[['1','2','3','4','5','6','7','8','9','10','11','12']] = pv_hydro_annual_stab_df[['1','2','3','4','5','6','7','8','9','10','11','12']].copy()

# Computing the yearly mean across the months
rank_c_stab_pv_hydro_df = pd.DataFrame()
rank_c_stab_pv_hydro_df['site_id'] = c_stab_pv_hydro_df['site_id']
rank_c_stab_pv_hydro_df['c_stab_mean'] = c_stab_pv_hydro_df[['1','2','3','4','5','6','7','8','9','10','11','12']].mean(axis=1)

In [ ]:
c_stab_pv_hydro_df

In [ ]:
rank_c_stab_pv_hydro_df.sort_values(by='c_stab_mean',ascending=True)

In [ ]:
# Calculating the deciles for the yearly mean
rank_c_stab_pv_hydro_df['decile_rank']= pd.qcut(rank_c_stab_pv_hydro_df['c_stab_mean'], 
                                         q = 10, labels = False)
rank_c_stab_pv_hydro_df.head()

In [ ]:
#rank_c_stab_pv_hydro_df.to_csv('rank.csv')

In [ ]:
coo_df = pv_wind_id_df[['lat','lon','site_id','name']]
input_df = pd.read_csv('../data/misc/combined_hydro_stream_gauge_full2.csv')
input_df_capacity = input_df[['stream_gauge_id','potential_cap_mw']].copy()


coo_df=coo_df.merge(input_df_capacity, left_on='site_id', right_on='stream_gauge_id', how='outer')
coo_df.head()

In [ ]:
rank_c_stab_pv_hydro_df=rank_c_stab_pv_hydro_df.merge(coo_df, left_on='site_id', right_on='site_id', how='outer')

In [ ]:
rank_c_stab_pv_hydro_df.head()

In [ ]:
rank_c_stab_pv_hydro_df.to_csv('../data/npd/complementarity/npd_pv_hydro_c_stab_decile_rank.csv')
rank_c_stab_pv_hydro_df.dropna(inplace=True)
rank_c_stab_pv_hydro_df

In [ ]:
how_many_on_each_decile = {}

for dec in rank_c_stab_pv_hydro_df.decile_rank.unique():
    how_many_on_each_decile[str(dec)] = rank_c_stab_pv_hydro_df.loc[rank_c_stab_pv_hydro_df.decile_rank == dec].count()


In [ ]:
import random

def pre_process_for_rank_plotting(rank_df, monthly_df):
    plotting_sites = []
    
    for dec in rank_df.decile_rank.unique():
        #print(dec)
        # Obtain the best site for each rank (mean)
        site = rank_df.loc[rank_df.decile_rank == dec].sort_values(by='c_stab_mean',ascending=False).site_id.values[0]
        print(f"Best site {site}, {rank_df.name.loc[rank_df.site_id == site].values[0]} for decile {dec} with stability coefficient {rank_df.c_stab_mean.loc[rank_df.site_id == site].values}")
        tmp_tup = (site,dec)
        plotting_sites.append(tmp_tup)
       
    # Create an empty dataframe to re-organize the data for plotting line ranking
    no_of_Rows = len(pv_hydro_annual_stab_df)
    plotting_df = pd.DataFrame(index=range(no_of_Rows))
    plotting_df['site_id'] = 0
    plotting_df['Month'] = 0
    plotting_df['c_stab'] = 0
    plotting_df['rank'] = 0
    
    plot_df_list = []
    months = range(1,13)
    for month in months:
        #print(month)
        for idx, row in monthly_df.iterrows():
            #print(row)
            plotting_df.loc[idx,'site_id'] = row['site_id']
            plotting_df.loc[idx,'Month'] = month
            plotting_df.loc[idx,'c_stab'] = row[str(month)]
            plotting_df.loc[idx,'decile_rank'] = rank_df.decile_rank.loc[rank_df.site_id == row['site_id']].values[0]

        plot_df_list.append(plotting_df.copy())
    
    # This is the dataframe with all the sites and ranks, reformatted for line plot
    c_stab_plotting_data = pd.concat(plot_df_list)
    
    # This is the dataset that will contain only the top site for each rank. In this case, 10 sites
    #c_stab_min_rank_plot = pd.DataFrame(index=len(tmp_tup))
    tmp_list = []
    for item in plotting_sites:
        tmp_df = c_stab_plotting_data.loc[plotting_df.site_id == item[0]].copy()
        tmp_df['rank'] = item[1]
        tmp_list.append(tmp_df)
    c_stab_min_rank_plot = pd.concat(tmp_list)
    
    return c_stab_plotting_data, c_stab_min_rank_plot
        
        

In [ ]:
plot_df, min_plot_df = pre_process_for_rank_plotting(rank_c_stab_pv_hydro_df, c_stab_pv_hydro_df)

In [ ]:
min_plot_df

In [ ]:
min_plot_df = min_plot_df.astype({"site_id": str})
min_plot_df.reset_index(inplace=True, drop=True)
min_plot_df.sort_values(by='rank', ascending=False,inplace=True)

In [ ]:
min_plot_df.head()

In [ ]:
sns.set()
rel = sns.relplot(
    data=min_plot_df,
    x="Month", y="c_stab",
    kind="line",hue="site_id",size="rank",
    height=7.5, aspect=1.5, facet_kws=dict(sharex=False),legend="full"
).set(title="The best NPDs for each decile in PV-Hydro complementarity")
#rel.fig.suptitle("The best NPDs for each decile")

1. take the average across the months for each site
2. rank the table based on the value of the average
3. bin this data to obtain "top" percentile (e.g. 20th)
4. assign a color to each bin (e.g. top 20th yellow,...). Use the same color scale that NREL uses
5. Use this in a map

In [ ]:
import seaborn as sns

#sns.set()
sns.set_theme(style="dark")
#sns.set_context("talk")
# Plot each year's time series in its own facet
g = sns.relplot(
    data=min_plot_df,
    x="Month", y="c_stab", col="site_id", hue="rank",
    kind="line", palette="crest", linewidth=4, zorder=5,
    col_wrap=5, height=4, aspect=1.5, legend=False,facet_kws=dict(despine=True)
)

# Iterate over each subplot to customize further
for site, ax in g.axes_dict.items():
    
    # Add the title as an annotation within the plot
    ax.text(.6, .75, f"NPD site: {site}", transform=ax.transAxes, fontweight="bold")
    
    # Add the title as an annotation within the plot
    rank = min_plot_df["rank"].loc[min_plot_df.site_id == site].values[0]
    ax.text(.8, .65, f"Rank: {rank}", transform=ax.transAxes, fontweight="bold")

    # Plot every year's time series in the background
    sns.lineplot(
        data=min_plot_df, x="Month", y="c_stab", units="site_id",
        estimator=None, color=".7", linewidth=1, ax=ax,
    )
    ax.set_ylim([0,1])

# Reduce the frequency of the x axis ticks
ax.set_xticks(ax.get_xticks()[::2])
# Tweak the supporting aspects of the plot
g.set_titles("")
g.set_axis_labels("Months", "Stability Coefficient")
#g.fig.suptitle("The best NPDs for each decile")
g.tight_layout()
#plt.savefig("ranked_pv_hydro_deciles.eps")

In [ ]:
g_plots = list(g.axes_dict.items())
g_plots

In [ ]:
for site, ax in g.axes_dict.items():
    
    # Add the title as an annotation within the plot
    ax.plot
    
    break

In [ ]:
rank_9_plot_df = plot_df.loc[plot_df.decile_rank == 9.0]

rank_9_plot_df = rank_9_plot_df.astype({"site_id": str})
rank_9_plot_df.reset_index(inplace=True, drop=True)

In [ ]:
rank_9_plot_df.sort_values(by='c_stab')

In [ ]:
g = sns.relplot(
    data=rank_9_plot_df,
    x="Month", y="c_stab",
    kind="line",hue="site_id",facet_kws=dict(despine=False)
)


### NPD Wind-Hydro

In [ ]:
wind_hydro_annual_stab_df = pd.read_csv(f"{c_stab_path}/rep_year_MONTHLY_npd_stab_wind_hydro.csv")
wind_hydro_annual_stab_df

# Cleaning up the dataframe
c_stab_wind_hydro_df = pd.DataFrame()
c_stab_wind_hydro_df['site_id'] = wind_hydro_annual_stab_df['site_id']
c_stab_wind_hydro_df[['1','2','3','4','5','6','7','8','9','10','11','12']] = wind_hydro_annual_stab_df[['1','2','3','4','5','6','7','8','9','10','11','12']].copy()

# Computing the yearly mean across the months
rank_c_stab_wind_hydro_df = pd.DataFrame()
rank_c_stab_wind_hydro_df['site_id'] = c_stab_wind_hydro_df['site_id']
rank_c_stab_wind_hydro_df['c_stab_mean'] = c_stab_wind_hydro_df[['1','2','3','4','5','6','7','8','9','10','11','12']].mean(axis=1)

In [ ]:
rank_c_stab_wind_hydro_df.sort_values(by='c_stab_mean',ascending=True)
# Calculating the deciles for the yearly mean
rank_c_stab_wind_hydro_df['decile_rank']= pd.qcut(rank_c_stab_wind_hydro_df['c_stab_mean'], 
                                         q = 10, labels = False)

plot_wind_hydro_df, min_wind_hydro_plot_df = pre_process_for_rank_plotting(rank_c_stab_wind_hydro_df, c_stab_wind_hydro_df)
min_plot_df = min_wind_hydro_plot_df.astype({"site_id": str})
min_plot_df.reset_index(inplace=True)

In [ ]:
min_wind_hydro_plot_df

In [ ]:
#coo_df = pv_wind_id_df[['lat','lon','site_id']]
#input_df = pd.read_csv('combined_hydro_stream_gauge_full2.csv')
#input_df_capacity = input_df[['stream_gauge_id','potential_cap_mw']].copy()

#coo_df=coo_df.merge(input_df_capacity, left_on='site_id', right_on='stream_gauge_id', how='outer')
#coo_df.head()

In [ ]:
#rank_c_stab_wind_hydro_df=rank_c_stab_wind_hydro_df.merge(coo_df, left_on='site_id', right_on='site_id', how='outer')
#rank_c_stab_wind_hydro_df.head()

In [ ]:
rank_c_stab_wind_hydro_df.to_csv('../data/npd/complementarity/npd_wind_hydro_c_stab_decile_rank.csv')

In [ ]:
import seaborn as sns

#sns.set()
sns.set_theme(style="dark")
#sns.set_context("talk")
# Plot each year's time series in its own facet
g = sns.relplot(
    data=min_plot_df,
    x="Month", y="c_stab", col="site_id", hue="rank",
    kind="line", palette="crest", linewidth=4, zorder=5,
    col_wrap=5, height=4, aspect=1.5, legend=False,
)

# Iterate over each subplot to customize further
for site, ax in g.axes_dict.items():
    
    # Add the title as an annotation within the plot
    ax.text(.6, .75, f"NPD site: {site}", transform=ax.transAxes, fontweight="bold")
    
    # Add the title as an annotation within the plot
    rank = min_plot_df["rank"].loc[min_plot_df.site_id == site].values[0]
    ax.text(.8, .65, f"Rank: {rank}", transform=ax.transAxes, fontweight="bold")

    # Plot every year's time series in the background
    sns.lineplot(
        data=min_plot_df, x="Month", y="c_stab", units="site_id",
        estimator=None, color=".7", linewidth=1, ax=ax,
    )
    ax.set_ylim([0,1])

# Reduce the frequency of the x axis ticks
ax.set_xticks(ax.get_xticks()[::2])
# Tweak the supporting aspects of the plot
g.set_titles("")
g.set_axis_labels("Months", "Stability Coefficient")
#g.fig.suptitle("The best NPDs for each decile")
g.tight_layout()